<a href="https://colab.research.google.com/github/Tinddd28/labs-iadnmo-/blob/main/lab_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 **Лабораторная работа № 5 "Классификация на основе обработки текстовой информации. Сверточные и рекуррентные нейронные сети. Полносвязные сети с предобучением и дообучением."**

# 1. Начало работы: импорт библиотек, подготовка данных

Импорт библиотек

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
import matplotlib.pyplot as plt

## Подготовка данных

Получение датасета, разделение на обучающую и валидационные выборки

In [5]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000)
# Преобразование данных к "плоскому" (numpy) массиву
x_train = sequence.pad_sequences(x_train, maxlen=500)
x_test = sequence.pad_sequences(x_test, maxlen=500)

# 2. Задание № 1. Создание двунаправленной LSTM рекуррентной сети

In [19]:
lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(
        input_dim=10000,
        output_dim=128,
        input_length=500,
    ),
    tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(
            units=64,
            activation="relu",
            return_sequences=False,
        ),
    ),
    tf.keras.layers.Dropout(0.4),

    tf.keras.layers.Dense(
        units=64,
        activation="relu",
    ),

    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Dense(
        units=1,
        activation="sigmoid",
    )
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [20]:
lstm_model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"],
)

In [16]:
callback_lstm = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_f1_score",
        mode="max",
        patience=5,
    ),
    tf.keras.callbacks.ModelCheckpoint(
        filepath="lstm_model.keras",
        monitor="val_f1_score",
        mode="max",
        save_best_only=True,
    ),
]

In [ ]:
history_lstm = lstm_model.fit(
    x_train,
    y_train,
    epochs=100,
    batch_size=64,
    validation_split=0.2,
    callbacks=callback_lstm,
)

Epoch 1/100
 38/313 ━━━━━━━━━━━━━━━━━━━━ 5:14 1s/step - accuracy: 0.5305 - loss: 0.6927